### 초기 명령어 모음

# 1. 환경 설정

### Qwen2.5-1.5B 모델 다운로드
> sudo apt install git-lfs \
> git clone https://huggingface.co/Qwen/Qwen2.5-1.5B

### 필요한 python 패키지 설치

In [ ]:
!pip install accelerate vllm==0.7.2 locust

# 2. local vLLM과 transformers 성능 비교

## transformers: 모델과 토크나이저 설정

In [ ]:
from time import time

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_path = "./models/Qwen2.5-1.5B-Instruct/"
model = AutoModelForCausalLM.from_pretrained(model_path, 
                                             device_map='auto',
                                             torch_dtype=torch.float16)

tokenizer = AutoTokenizer.from_pretrained(model_path)

## transformers: 기본 생성

In [ ]:
prompt = "Hello, my name is"
input_tokens = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to(model.device)

start_time = time()
output_tokens = model.generate(**input_tokens, 
                               max_new_tokens=200, 
                               temperature=0.8,
                               top_p=0.95,
                              )
output = tokenizer.decode(output_tokens[0])
print(output)
end_time = time()
print("Time spent to generate:", end_time - start_time)

## transformers: 스트리밍 생성

In [ ]:
from transformers import TextStreamer
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

prompt = "Hello, my name is"
print(prompt, end='')
input_tokens = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to(model.device)

start_time = time()
output_tokens = model.generate(**input_tokens, 
                               max_new_tokens=200, 
                               temperature=0.8,
                               top_p=0.95,
                               streamer=streamer)
end_time = time()
print("Time spent to generate:", end_time - start_time)

## local vLLM: 기본 생성

In [ ]:
from vllm import LLM, SamplingParams

model_path = "./models/Qwen2.5-1.5B-Instruct/"
llm = LLM(model=model_path, 
          dtype='half',
          gpu_memory_utilization=0.6,
          enforce_eager=True
         )

In [ ]:
prompt = "Hello, my name is"

sampling_params = SamplingParams(
    temperature=0.8,
    top_p=0.95,
    max_tokens=200,
    # stream=True
)

outputs = llm.generate([prompt], sampling_params)
for output in outputs:
    print(output.outputs[0].text, end="", flush=True)

# 3. vLLM을 이용한 API server 띄우기

- 터미널 탭을 추가로 띄워서 실행
- 다운로드 받은 모델로 vLLM 서빙 띄우는 명령어:
```bash
$ vllm serve models/Qwen2.5-1.5B-Instruct/ \
  --port 8006 \
  --dtype half \
  --gpu-memory-utilization 0.6 \
  --enforce-eager
```

- 입력변수 설명
    - `--gpu-memory-utilization`: 초기에 vllm에서 잡고 가는 GPU 메모리 비율
    - `--enforce-eager`: pytorch에서 CUDA graph 세팅 기능을 끄는 입력변수
        - CUDA graph 세팅 시 속도↑ & 기본 GPU 메모리 사용량 증가↑

- 우리 환경에서 추가 실험 가능한 arguments:
    - `--max-model-len`: 모델의 입력+출력 최대 길이 설정
    - `--enable-prefix-caching` or `--no-enable-prefix-caching`: prefix caching의 on-off